# Encoder-Decoderモデルを書く

ニューラル機械翻訳用のEncoder-DecoderモデルをChainerで書いてみます。  
[公式のexample](https://github.com/chainer/chainer/blob/master/examples/seq2seq/seq2seq.py)を参考にしましたので、そちらも見てみてください。  

## ライブラリのimport

In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

## ニューラル機械翻訳（NMT)

NMTにおけるEncoder-Decodeモデルは、主に二つのコンポーネントを持っています。  

- Encoder: ソース言語の文を、ベクトルに変換する。
- Decoder: Encoderの出力をターゲット言語の文に変換する。

以下では、それぞれ分けて書いていきます。

まず、可変長のid系列のリストをembeddingの系列に変換する関数`sequence_embed`を用意しましょう。

In [2]:
# 可変長単語id系列を可変長単語ベクトル系列へ写像する関数
def sequence_embed(embed, xs):
    x_len = [len(x) for x in xs]
    x_section = np.cumsum(x_len[:-1])
    ex = embed(F.concat(xs, axis=0))
    exs = F.split_axis(ex, x_section, 0)
    return exs

### Encoder

Encoderモデルを書きます。  
入力単語系列をembeddingの系列に変換し、それらをNStepLSTMでencodeします。

In [1]:
class Encoder(Chain):
    def __init__(self, n_layers, n_source_vocab, n_units, dropout):
        super(Encoder, self).__init__()
        with self.init_scope():
            self.source_embed = L.EmbedID(in_size=n_source_vocab, out_size=n_units)
            self.encoder_lstm = L.NStepLSTM(n_layers=n_layers, in_size=n_units,
                                            out_size=n_units, dropout=dropout)
            self.n_source_vocab = n_source_vocab
            self.n_units = n_units
    
    def __call__(self, source_xs):
        # 単語の系列を単語ベクトルへ
        exs = sequence_embed(self.source_embed, source_xs)
        
        # lstmの初期状態
        hx = None
        cx = None
        
        # lstmで各系列をエンコード
        hy, cy, ys = self.encoder_lstm(hx, cx, exs)
        return hy, cy, ys
        

NameError: name 'Chain' is not defined

### Decoder

次にDecoderを書きます。  
DecoderもEmbedIDとNStepLSTMで書いて、あとでEncoder-Decodeモデルとしてまとめたクラスを書くことにします。

In [9]:
class Decoder(Chain):
    def __init__(self, n_layers, n_target_vocab, n_units, dropout):
        super(Decoder, self).__init__()
        with self.init_scope():
            self.target_embed = L.EmbedID(in_size=n_target_vocab, out_size=n_units)
            self.decoder_lstm = L.NStepLSTM(n_layers=n_layers, in_size=n_units,
                                            out_size=n_units, dropout=dropout)
            self.n_target_vocab = n_target_vocab
            self.n_units = n_units
        
    def __call__(self, hy, cy, target_xs):
        # targetの単語系列を単語ベクトルへ
        exs = sequence_embed(self.target_embed, target_xs)
        
        # encoderの出力を受け取り、lstmでデコード
        ho, co, os = self.decoder_lstm(hy, cy, exs)
        
        # 各タイムステップのhidden layerを返す
        return ho, co, os

### Encoder-Decoderモデル

EncoderとDecoderを書いたので、それらをつなげます。

NMTではEncoder-Decoderをいきなりひとつのモデルで定義しても問題ないと思いますが、今回はモデルを分けて保存・再利用できるようにするために、別々のクラスで書きました。

学習時のloss計算用のメソッドと予測時の翻訳用のメソッドを書くことにします。  


In [5]:
# UNKとEOSのidを指定しておく
UNK = 0
EOS = 1

In [10]:
class Encoder_Decoder(Chain):
    def __init__(self, encoder, decoder):
        super(Encoder_Decoder, self).__init__()
        with self.init_scope():
            # EncoderとDecoderを引数で受け取るようにしておく。
            self.encoder = encoder
            self.decoder = decoder
            
            # Decoderの隠れ状態からtargetボキャブラリーを予測する線形変換用のレイヤー
            self.W = L.Linear(self.decoder.n_target_vocab)
            # self.xp にはnumpyかcupyが勝手に入る
    
    def __call__(self, xs, ys):
        # xs: sourceの単語idの系列のリスト
        # ys: targetの単語idの系列のリスト
        
        # EOS=1
        # targetの単語sequenceにEOSをくっつける。
        # 学習時のDecoderの入力においては系列の前に(ys_in)
        # 答え合わせの時は系列の後に(ys_out)
        eos = self.xp.array([EOS], 'i')
        ys_in = [F.concat([eos, y], axis=0) for y in ys]
        ys_out = [F.concat([y, eos], axis=0) for y in ys]
        
        # encode & decode
        hy, cy, _ = self.encoder(xs)
        _, _, os = self.decoder(hy, cy, ys_in)
        
        # loss calculation
        batch_size = len(xs)
        concat_os = F.concat(os, axis=0)
        concat_ys_out = F.concat(ys_out, axis=0)
        loss = F.sum(F.softmax_cross_entropy(
            self.W(concat_os), concat_ys_out, reduce='no')) / batch_size
        
        return loss
    
    def translate(self, xs, max_length = 100):
        batch_size = len(xs)
        
        # 予測なので、backpropやdropoutをしないモードでモデルを動かす
        with chainer.no_backprop_mode(), chainer.using_config('train', False):
            # sourceの系列のエンコード
            h, c, _ = self.encoder(xs)
            
            # decode時のinput用にbatch_size分のEOS=1を用意
            ys = self.xp.full((batch_size,1), 1, 'i')
            result = []
            
            for i in range(max_length):
                h, c, ys = self.decoder(h, c, ys)
                cys = F.concat(ys, axis=0)
                wy = self.W(cys)
                ys = self.xp.argmax(wy.data, axis=1).astype('i')
                result.append(ys)
                ys = F.reshape(ys, (-1, 1)).data
            
            result = self.xp.stack(result).T
            # Remove EOS tags
            outs = []
            for y in result:
                inds = np.argwhere(y == EOS)
                if len(inds) > 0:
                    y = y[:inds[0, 0]]
                outs.append(y)
        return outs